# MNIST with ANN

### Imports

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix

In [ ]:
import torch
import torch.nn as nn
import torch.